In [1]:
import pandas as pd
import warnings
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
raw_df=pd.read_csv('dataset.csv')

In [ ]:
high_revenue=['DLJ','MGK','XGS','XGU','ZXM']

# 1. Data Analysis

## 1.1 General analysis

Let's start by prioritizing main columns in our dataset, analywing them, check for null values, and overall data types.

In [3]:
main_columns = ['FlownMonth', 'ProductCode', 'AgentCode', 'AgentName']

numeric_columns = raw_df.select_dtypes(include=['number']).columns

# Prioritize main columns
df = raw_df.loc[:, raw_df.columns.isin(main_columns) | raw_df.columns.isin(numeric_columns)]

# Get price column (target regression variable)
df['PricePerWeight']=df['Revenue']/df['ChargeableWeight']

# Get basic information about my dataset
df.info()

print()
print()

print('number of duplicated rows:', df.shape[0]-df.drop_duplicates().shape[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7420 entries, 0 to 7419
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FlownYear         7420 non-null   int64  
 1   FlownMonth        7420 non-null   object 
 2   AgentCode         7420 non-null   object 
 3   AgentName         7420 non-null   object 
 4   ProductCode       7420 non-null   object 
 5   ChargeableWeight  7420 non-null   float64
 6   Pieces            7420 non-null   int64  
 7   Revenue           7420 non-null   float64
 8   PricePerWeight    7420 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 521.8+ KB


number of duplicated rows: 901


In [4]:
# Unique categories for main categorical variables columns
print('#==================== categorical variables =========================#')

for column in main_columns:
    print(f'number of unique {column}:', df[column].nunique())

print()
print()

# Distribution of numerical variables
print('#================ numerical variables variables =====================#')

for column in numeric_columns:
    print(f'Statistics of {column} column:', df[column].describe())
    print()


print()


#==================== categorical variables =========================#
number of unique FlownMonth: 3
number of unique ProductCode: 18
number of unique AgentCode: 679
number of unique AgentName: 664


#================ numerical variables variables =====================#
Statistics of FlownYear column: count    7420.0
mean     2017.0
std         0.0
min      2017.0
25%      2017.0
50%      2017.0
75%      2017.0
max      2017.0
Name: FlownYear, dtype: float64

Statistics of ChargeableWeight column: count      7420.000000
mean       2276.926132
std        5991.667675
min           0.300000
25%         177.000000
50%         639.000000
75%        1998.000000
max      177078.000000
Name: ChargeableWeight, dtype: float64

Statistics of Pieces column: count    7420.000000
mean       20.956873
std        67.731589
min         1.000000
25%         1.000000
50%         5.000000
75%        14.000000
max      2000.000000
Name: Pieces, dtype: float64

Statistics of Revenue column: count    7.4200

We can note that there are no problems with empty values, however, we have around 1K duplicated rows. This will be further considered in the modelling phase. In addition, we can note:

 1. There are revenue values equal to 0, which might not be convienient to keep given that the target variable to predict is computed based on the revenue, so we will ignore these 3 rows.

 2. Since the year column contains one single value, it is not really informative, so we can as well omit it. 

Now, let's analyze the distributions of our features separately.

In [5]:
df=df[df['Revenue']>0] ## remove zero revenue values

df=df.loc[:,df.columns.drop('FlownYear')] ## drop flown year

In [6]:
from utils import format_plot

numeric_columns = df.select_dtypes(include=['number']).columns

## distribution of numeric columns
for variable in numeric_columns:

    fig_variable = px.histogram(df, x=variable, nbins=50, marginal="box", title=f'{variable} Distribution')


    formatted_fig_variable = format_plot(fig_variable, 
                                        title=f'{variable} Distribution', 
                                        xaxis_title=variable, 
                                        yaxis_title='Count')

    formatted_fig_variable.show()


## distribution of categorical columns
categorical_columns = df.select_dtypes(include=['category', 'object']).columns

for variable in categorical_columns:

    frequency=df[variable].value_counts().reset_index()

    fig_variable = px.bar(frequency, x=frequency[variable],y=frequency['count'], title=f'{variable} Distribution')


    formatted_fig_variable = format_plot(fig_variable, 
                                        title=f'{variable} Distribution', 
                                        xaxis_title=variable, 
                                        yaxis_title='Count')

    formatted_fig_variable.show()

From the plots the main insights we can appreciate, which will have an impact during the modelling phase is that the distribution of the numerical variables is highly skewed to the left. In addition, there are over-represented products such as the DLJ and MGK one.

In [7]:
from utils import plot_grouped_bar_chart

product_data = df.groupby('ProductCode').agg(
    total_products=('ProductCode', 'count'),  # Change to mean instead of sum
    total_revenue=('Revenue', 'sum')
).reset_index()

plot_grouped_bar_chart(
    df=product_data,
    x_col='ProductCode', 
    y_col1='total_products', 
    y_col2='total_revenue', 
    title='Total Products and Revenue by Product Code'
)

We can note how the total revenue is dominated by a few product codes, particularly DLJ and MGK, 
which were the most shipped ones.

These products should defenitely be a business priority, 
and the ML pipelines should guarantee good predictions for them.      

#### 1.2 Target variable

In [8]:
df['PricePerWeight'].describe()

count    7417.000000
mean       13.020357
std        70.102431
min         0.080000
25%         0.415839
50%         0.630012
75%         2.750000
max      4329.333333
Name: PricePerWeight, dtype: float64

In [9]:
fig_price_per_weight = px.histogram(df, x='PricePerWeight', nbins=50, marginal="box", title='Price Per Weight Distribution')

formatted_fig_price_per_weight = format_plot(fig_price_per_weight, 
                                            title='Price Per Weight Distribution', 
                                            xaxis_title='Price Per Weight', 
                                            yaxis_title='Count')


formatted_fig_price_per_weight.show()


df['LogPricePerWeight'] = np.log1p(df['PricePerWeight'])  # log1p ensures that you handle zero values safely.

q_hi  = df["LogPricePerWeight"].quantile(0.95)

df_filtered = df[(df["LogPricePerWeight"] < q_hi)]


fig_log_price_per_weight = px.histogram(df_filtered, x='LogPricePerWeight', nbins=50, marginal="box", title='Log Price Per Weight Distribution')

formatted_fig_log_price_per_weight = format_plot(fig_log_price_per_weight, 
                                                title='Log Price Per Weight Distribution', 
                                                xaxis_title='Log Price Per Weight', 
                                                yaxis_title='Count')

formatted_fig_log_price_per_weight.show()


We can notice that the skewness from revenue and chargeable weight was propagated to the 'PricePerWeight' variable, which also exhibits high variability. 

Approximately 75% of the prices are equal to or less than 2.75, but there is a significant spread, with a standard deviation of 70. 
This suggests that 20% of the values are much larger than the rest.

For traditional linear models, this high skewness can be problematic, as they may overly focus on outliers or the larger tail, 
resulting in poor generalization on more typical data points. 
This shows the need of implementating models more robust to skewed distributions, such as tree-based ones.
      
Finally, performing the log transformation we can better visualize the distribution.

## 1.3 Main features vs. Target variable

In [10]:
from utils import plot_beautiful_corr_heatmap

plot_beautiful_corr_heatmap(df, numeric_columns)

As expected, the revenue shows the highest, but moderate positive correlation (0.43) 
with price per weight, while other linear relationships, are weak. The number of pieces and 
chargeable weight have a moderate positive correlation, 
suggesting that they increase together, but overall, most features operate independently. 
Non-linear relationships could still hold.

In [11]:
fig = px.box(df_filtered, x='ProductCode', y='LogPricePerWeight', 
             color='ProductCode',
             template="plotly_white")

fig.update_layout(
    title=dict(
        text="Price Per Weight Distribution by Product Code",
        x=0.5,  
        xanchor='center'
    ),
    showlegend=False)

fig.show()

We observed that MGK and DLJ are the main revenue contributors, as they dominate both total shipments and revenue. 
At the same time, these two product codes also exhibit the highest price per weight outliers, as seen in the distribution plot.
This combination of high revenue contribution and significant price variability suggests that pricing for these products is 
not only inconsistent but also prone to extreme values. For this reason, it becomes even more relevant to propose a robust modeling
approach that can effectively handle outliers and the skewed distribution.

In [12]:
fig = px.box(df_filtered.sort_values(by=['FlownMonth'],ascending=False), x='FlownMonth', y='LogPricePerWeight', 
             title="Price Per Weight Distribution by Month",
             template="plotly_white")

fig.update_layout(
    title=dict(
        text="Price Per Weight Distribution by by Month",
        x=0.5,  
        xanchor='center'
    ),
    showlegend=False,  

)

fig.show()

The distribution of log-transformed price per weight remains consistent across September, October, and November,
with similar medians and interquartile ranges.
However, there are significant outliers in all months, indicating consistent price per weight outliers accross months.